In [ ]:
import geopandas as gpd
import pandas as pd
import folium as fm
from folium.plugins import MarkerCluster

In [ ]:
df = gpd.read_file('sample_data_loc2018.csv')
df

In [ ]:
# Since we don't know what the columns mean, we can't plot assumptions...
# Assumption:
    # 'tway' stands for "type of way" or roadway classification.
    # 'lat' is the latitude
    # 'lon' is the longitude

# Make an interactive map using Folium
# Note, since geopandas doesn't automatically detect the geometry from the data, we need to pre-process the data. 

In [ ]:
# Load the data
df = pd.read_csv('sample_data_loc2018.csv')

# Convert lat and lon to correct decimal degrees
df['lat'] = df['lat'] / 1000000
df['lon'] = - (df['lon'] / 1000000)  # Note the negative sign for longitude

# Remove rows with invalid lat/lon
df = df.dropna(subset=['lat', 'lon'])

# Create a map centered on South Carolina
sc_center_lat, sc_center_lon = 33.8361, -81.1637  # Approximate center of SC
m = fm.Map(location=[sc_center_lat, sc_center_lon], zoom_start=7)

# Create a MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

# Define color mapping for 'tway'
color_map = {1: 'red', 2: 'blue', 3: 'green', 4: 'purple', 5: 'orange'}

# Add markers to the cluster
for idx, row in df.iterrows():
    fm.Marker(
        location=[row['lat'], row['lon']],
        popup=f"Type: {row['tway']}, Road: {row['als']}",
        icon=fm.Icon(color=color_map.get(row['tway'], 'gray'))
    ).add_to(marker_cluster)

# Add a legend
legend_html = '''
<div style="position: fixed; bottom: 20px; left: 50px; width: 100px; height: 175px; 
    border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
    ">&nbsp; <b> Type of Way </b><br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; Type 1 <br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i>&nbsp; Type 2 <br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Type 3 <br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:purple"></i>&nbsp; Type 4 <br>
    &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i>&nbsp; Type 5 <br>
</div>
'''
m.get_root().html.add_child(fm.Element(legend_html))

# Add borders to the map for South Carolina
# Source: https://nagasudhir.blogspot.com/2021/07/draw-borders-from-geojson-paths-in.html
# style options - https://leafletjs.com/reference-1.7.1.html#path
bordersStyle = {
    'color': 'green',
    'weight': 2,
    'fillColor': 'blue',
    'fillOpacity': 0.1
}

# File (`south carolina.geojson`) downloaded from https://github.com/glynnbird/usstatesgeojson/blob/master/south%20carolina.geojson
fm.GeoJson(
    data=(open("south carolina.geojson", 'r').read()),
    name="South Carolina",
    style_function=lambda x: bordersStyle).add_to(m)

# Save the map
m.save("south_carolina_traffic_incidents_map.html")

print("Map has been saved as 'south_carolina_traffic_incidents_map.html'")

In [ ]:
m